<h1 style="text-align:center;">ESCUELA POLITÉCNICA NACIONAL</h1>
<h2 style="text-align:center;">FACULTAD DE INGENIERÍA DE SISTEMAS</h2>
<h3 style="text-align:center;">INGENIERÍA DE COMPUTACIÓN</h3>

**PERÍODO ACADÉMICO:** 2025-B

**ASIGNATURA:** ICCD412 Métodos Numéricos  

**GRUPO:** GR2  

**TIPO DE INSTRUMENTO:** Proyecto bimestral  

**FECHA DE ENTREGA LÍMITE:**   

**ALUMNOS:** Guerra Melissa, Velastegui Joel

# LIBRERÍAS

##### Se utilizarán las librerás pandas y numpy, para poder trabajar con la base de datos y aplicar los métodos numéricos que vamos a emplear.

In [1]:
import numpy
import pandas

# IMPLEMENTACIÓN

In [6]:
import pandas as pd
import numpy as np


def interpolar_dataset_remmaq(
    ruta_excel,
    hora_inicio_dia=6,
    hora_fin_dia=18,
    columnas_modelo=None,
    nombre_salida="dataset_interpolado"
):
    """
    Interpola datos horarios de calidad del aire REMMAQ siguiendo criterios físicos.

    Criterios aplicados:
    - Interpolación temporal lineal para variables continuas
    - Radiación solar (RS) se interpola solo durante el día
    - Valores 0 se mantienen (mediciones válidas)
    - Valores NA se interpolan cuando es posible
    - Filas con datos incompletos finales se eliminan

    Parámetros
    ----------
    ruta_excel : str
        Ruta del archivo Excel con los datos REMMAQ.
    hora_inicio_dia : int
        Hora de inicio del periodo diurno (por defecto 6).
    hora_fin_dia : int
        Hora de fin del periodo diurno (por defecto 18).
    columnas_modelo : list[str], opcional
        Lista de columnas usadas en el modelo.
        Si es None, se usan las variables estándar del proyecto.
    nombre_salida : str
        Nombre base de los archivos de salida (sin extensión).

    Retorna
    -------
    pd.DataFrame
        DataFrame interpolado y listo para modelado numérico.
    """

    # -----------------------------
    # 1. Cargar datos
    # -----------------------------
    df = pd.read_excel(ruta_excel)

    df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
    df = df.sort_values("Fecha").set_index("Fecha")

    # -----------------------------
    # 2. Definición de variables
    # -----------------------------
    if columnas_modelo is None:
        columnas_modelo = ["CO", "VEL", "TMP", "RS"]

    vars_interpolar = ["CO", "VEL", "TMP"]
    col_rs = "RS"

    # -----------------------------
    # 3. Interpolación temporal general
    # -----------------------------
    df[vars_interpolar] = df[vars_interpolar].interpolate(
        method="time",
        limit_direction="both"
    )

    # -----------------------------
    # 4. Interpolación física de RS
    # -----------------------------
    hora = df.index.hour
    es_dia = (hora >= hora_inicio_dia) & (hora <= hora_fin_dia)

    rs_aux = df[col_rs].copy()
    rs_aux[es_dia] = rs_aux[es_dia].interpolate(
        method="time",
        limit_direction="both"
    )

    df[col_rs] = rs_aux

    # -----------------------------
    # 5. Limpieza final
    # -----------------------------
    df_final = df.dropna(subset=columnas_modelo)

    # -----------------------------
    # 6. Guardar resultados
    # -----------------------------
    df_final.to_csv(f"{nombre_salida}.csv")
    df_final.to_excel(f"{nombre_salida}.xlsx")

    return df_final


In [8]:
df_interpolado = interpolar_dataset_remmaq(
    ruta_excel="ProyMetodos_DatosCentro.xlsx",
    nombre_salida="datos_interpolados_centro"
)
